# Défi TextMine 2025 : Lecture les données

## Chargement

1. On charge le CSV avec `pandas`.
2. On parse les colonnes "entities" et "relations" avec `json`.

In [1]:
import pandas as pd
import json

train_df = pd.read_csv("train.csv") # chargement des données d'entraînement depuis Kaggle
train_df = train_df.set_index("id")
train_df.entities = train_df.entities.apply(json.loads) # parsing des entités
train_df.relations = train_df.relations.apply(json.loads) # parsing des relations
train_df.head()

,text,entities,relations
id,,,
181,"Anam Destresse, président de l'ONG ""Ma passion...","[{'id': 0, 'mentions': [{'value': 'accident', ...","[[0, STARTED_IN, 9], [7, IS_LOCATED_IN, 9], [5..."
31669,"À Paris, le 8 avril 2022, l'usine de déodorant...","[{'id': 0, 'mentions': [{'value': 'explosé', '...","[[9, IS_LOCATED_IN, 8], [11, OPERATES_IN, 8], ..."
51470,"En Espagne, dans une région agricole, une cont...","[{'id': 0, 'mentions': [{'value': 'contaminati...","[[7, IS_PART_OF, 8], [9, OPERATES_IN, 1], [0, ..."
51332,Un important incendie a fait des ravages dans ...,"[{'id': 0, 'mentions': [{'value': 'incendie', ...","[[12, IS_IN_CONTACT_WITH, 5], [0, IS_LOCATED_I..."
1131,« Je coule » : onze heures après avoir envoyé ...,"[{'id': 0, 'mentions': [{'value': 'renversé', ...","[[9, IS_LOCATED_IN, 2], [0, START_DATE, 17], [..."


## Manipulation

Les entités liées à un texte sont décrites par des dictionnaires stockés dans une liste.

In [2]:
train_df.loc[181].text 

'Anam Destresse, président de l\'ONG "Ma passion", a été blessé dans un accident. Le 30 juin 2022, un accident de circulation s\'est produit entre une moto et un bus sur l\'autoroute de Saint-Marin en Italie. Le bus, qui transportait 20 passagers, appartenait à l\'ONG. Lors de l\'accident, les panneaux de signalisation ont été complètement endommagés et le garde du corps a été blessé. Au total, deux passagers sont morts sur le coup. Anam Destresse, qui faisait partie des blessés, a été transporté en hélicoptère jusqu\'à l\'hôpital. Le conducteur de la moto a été retrouvé mort en dessous du bus. Il conduisait sans permis et en état d\'ivresse.'

In [4]:
text_181_entity_0 = train_df.loc[181].entities[9] # sélection de la première entité listée pour le texte 181
text_181_entity_0

{'id': 9,
 'mentions': [{'value': 'autoroute de Saint-Marin', 'start': 169, 'end': 193}],
 'type': 'PLACE'}

In [3]:
text_181_entity_0 = train_df.loc[181].entities[0] # sélection de la première entité listée pour le texte 181
print(text_181_entity_0) # affichage de la description complète de l'entité
print(text_181_entity_0["mentions"][0]) # affichage de la description de la première mention de l'entité
print(text_181_entity_0["mentions"][0]["value"]) # affichage de la valeur de l'entité

{'id': 0, 'mentions': [{'value': 'accident', 'start': 70, 'end': 78}, {'value': 'accident de circulation', 'start': 100, 'end': 123}, {'value': 'accident', 'start': 275, 'end': 283}], 'type': 'ACCIDENT'}
{'value': 'accident', 'start': 70, 'end': 78}
accident


Les relations liées à un texte sont décrites par des triplets stockés dans une liste.

In [4]:
text_181_relation_0 = train_df.loc[181].relations[0] # sélection de la première relation listée pour le texte 181
print(text_181_relation_0) # affichage de la description complète de la relation
print(text_181_relation_0[0]) # affichage de l'entité source de la relation
print(text_181_relation_0[1]) # affichage du type de la relation
print(text_181_relation_0[2]) # affichage de l'entité cible de la relation

[0, 'STARTED_IN', 9]
0
STARTED_IN
9


In [5]:
example = train_df.loc[181]

print(example.text)
print(example.entities)
print(example.relations)

Anam Destresse, président de l'ONG "Ma passion", a été blessé dans un accident. Le 30 juin 2022, un accident de circulation s'est produit entre une moto et un bus sur l'autoroute de Saint-Marin en Italie. Le bus, qui transportait 20 passagers, appartenait à l'ONG. Lors de l'accident, les panneaux de signalisation ont été complètement endommagés et le garde du corps a été blessé. Au total, deux passagers sont morts sur le coup. Anam Destresse, qui faisait partie des blessés, a été transporté en hélicoptère jusqu'à l'hôpital. Le conducteur de la moto a été retrouvé mort en dessous du bus. Il conduisait sans permis et en état d'ivresse.
[{'id': 0, 'mentions': [{'value': 'accident', 'start': 70, 'end': 78}, {'value': 'accident de circulation', 'start': 100, 'end': 123}, {'value': 'accident', 'start': 275, 'end': 283}], 'type': 'ACCIDENT'}, {'id': 1, 'mentions': [{'value': 'Anam Destresse', 'start': 0, 'end': 14}, {'value': 'Anam Destresse', 'start': 431, 'end': 445}], 'type': 'CIVILIAN'}, 

## Statistiques du dataset

#### Pour les entités

In [6]:
len(train_df)

800

In [7]:
# Initialisation des compteurs pour les entités
entity_counts = {}
total_entities = 0

# Comptabiliser les types d'entités et le nombre total d'entités
for row in train_df['entities']:
    for entity in row:
        entity_type = entity['type']
        if entity_type in entity_counts:
            entity_counts[entity_type] += len(entity['mentions'])
        else:
            entity_counts[entity_type] = len(entity['mentions'])
        total_entities += 1

# Calcul de la moyenne des entités par document
average_entities_per_document = total_entities / len(train_df)

# Calcul du nombre total de types d'entités
total_entity_types = len(entity_counts)

# Trier les types d'entités par fréquence décroissante
sorted_entity_counts = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)

# Affichage des résultats
print("Nombre de fois que chaque type d'entité apparaît dans tout le dataset (ordre décroissant) :")
for entity_type, count in sorted_entity_counts:
    print(f"{entity_type}: {count} fois")

print(f"\nNombre total d'entités dans tout le dataset : {total_entities}")
print(f"Nombre moyen d'entités par document : {average_entities_per_document:.2f}")
print(f"Nombre total de types d'entités : {total_entity_types}")

Nombre de fois que chaque type d'entité apparaît dans tout le dataset (ordre décroissant) :
MATERIEL: 4546 fois
PLACE: 3116 fois
CIVILIAN: 2900 fois
GROUP_OF_INDIVIDUALS: 2768 fois
FIRSTNAME: 1177 fois
LASTNAME: 1157 fois
NON_GOVERNMENTAL_ORGANISATION: 1057 fois
CATEGORY: 886 fois
NON_MILITARY_GOVERNMENT_ORGANISATION: 831 fois
TIME_EXACT: 741 fois
ACCIDENT: 559 fois
FIRE: 372 fois
GATHERING: 356 fois
QUANTITY_FUZZY: 348 fois
TIME_FUZZY: 273 fois
CBRN_EVENT: 258 fois
TERRORIST_OR_CRIMINAL: 244 fois
QUANTITY_EXACT: 229 fois
NATIONALITY: 174 fois
EPIDEMIC: 137 fois
NATURAL_EVENT: 134 fois
THEFT: 134 fois
ECONOMICAL_CRISIS: 125 fois
CRIMINAL_ARREST: 100 fois
RIOT: 92 fois
COLOR: 89 fois
BOMBING: 79 fois
STRIKE: 70 fois
POLLUTION: 68 fois
DEMONSTRATION: 67 fois
MILITARY_ORGANISATION: 60 fois
ILLEGAL_CIVIL_DEMONSTRATION: 60 fois
POLITICAL_VIOLENCE: 58 fois
SUICIDE: 55 fois
TIME_MIN: 52 fois
TRAFFICKING: 52 fois
COUP_D_ETAT: 49 fois
QUANTITY_MIN: 44 fois
ELECTION: 42 fois
AGITATING_TROUBLE_MA

#### Pour les relations

In [8]:
# Initialisation des compteurs
relation_counts = {}
total_relations = 0

# Comptabiliser les types de relations et le nombre total de relations
for row in train_df['relations']:
    for relation in row:
        relation_type = relation[1]
        if relation_type in relation_counts:
            relation_counts[relation_type] += 1
        else:
            relation_counts[relation_type] = 1
        total_relations += 1

# Calcul de la moyenne des relations par document
average_relations_per_document = total_relations / len(train_df)

# Calcul du nombre total de types de relations
total_relation_types = len(relation_counts)

# Trier les types de relations par fréquence décroissante
sorted_relation_counts = sorted(relation_counts.items(), key=lambda x: x[1], reverse=True)

# Affichage des résultats
print("Nombre de fois que chaque type de relation apparaît dans tout le dataset (ordre décroissant) :")
for relation_type, count in sorted_relation_counts:
    print(f"{relation_type}: {count} fois")

print(f"\nNombre total de relations dans tout le dataset : {total_relations}")
print(f"Nombre moyen de relations par document : {average_relations_per_document:.2f}")
print(f"Nombre total de types de relations : {total_relation_types}")

Nombre de fois que chaque type de relation apparaît dans tout le dataset (ordre décroissant) :
IS_LOCATED_IN: 9025 fois
HAS_CONTROL_OVER: 4547 fois
IS_IN_CONTACT_WITH: 2919 fois
OPERATES_IN: 2435 fois
STARTED_IN: 1860 fois
IS_AT_ODDS_WITH: 1526 fois
IS_PART_OF: 1462 fois
START_DATE: 1034 fois
GENDER_MALE: 908 fois
HAS_CATEGORY: 894 fois
END_DATE: 874 fois
HAS_CONSEQUENCE: 769 fois
INITIATED: 469 fois
IS_OF_SIZE: 438 fois
GENDER_FEMALE: 414 fois
IS_COOPERATING_WITH: 372 fois
RESIDES_IN: 222 fois
HAS_FAMILY_RELATIONSHIP: 222 fois
HAS_QUANTITY: 191 fois
IS_OF_NATIONALITY: 179 fois
CREATED: 126 fois
HAS_COLOR: 91 fois
DEATHS_NUMBER: 75 fois
INJURED_NUMBER: 70 fois
IS_DEAD_ON: 68 fois
IS_BORN_IN: 50 fois
WEIGHS: 41 fois
DIED_IN: 41 fois
IS_REGISTERED_AS: 34 fois
IS_BORN_ON: 20 fois
HAS_FOR_LENGTH: 16 fois
WAS_CREATED_IN: 15 fois
HAS_FOR_WIDTH: 14 fois
WAS_DISSOLVED_IN: 14 fois
HAS_FOR_HEIGHT: 12 fois
HAS_LONGITUDE: 12 fois
HAS_LATITUDE: 10 fois

Nombre total de relations dans tout le datase

#### Nombre de phrases dans les textes

In [9]:
import spacy

# Charger le modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")


# Fonction pour compter le nombre de phrases avec spaCy
def count_sentences_spacy(text):
    doc = nlp(text)
    return len(list(doc.sents))

# Appliquer la fonction à la colonne 'text' pour obtenir le nombre de phrases dans chaque texte
train_df['sentence_count'] = train_df['text'].apply(count_sentences_spacy)

# Calcul des statistiques
max_sentences = train_df['sentence_count'].max()
min_sentences = train_df['sentence_count'].min()
median_sentences = train_df['sentence_count'].median()
mean_sentences = train_df['sentence_count'].mean()

# Afficher les résultats
print(f"Nombre max de phrases : {max_sentences}")
print(f"Nombre min de phrases : {min_sentences}")
print(f"Nombre médian de phrases : {median_sentences}")
print(f"Nombre moyen de phrases : {mean_sentences}")


/home/apeuvot/anaconda3/envs/textmine/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/apeuvot/anaconda3/envs/textmine/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/apeuvot/anaconda3/envs/textmine/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Nombre max de phrases : 15
Nombre min de phrases : 2
Nombre médian de phrases : 7.0
Nombre moyen de phrases : 7.0075


##### Tokens

In [1]:
import pandas as pd
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-large')

/home/apeuvot/anaconda3/envs/textmine/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# train_df = pd.read_csv("test_01-07-2024.csv") # chargement des données d'entraînement depuis Kaggle
# train_df = train_df.set_index("id")
# train_df.entities = train_df.entities.apply(json.loads) # parsing des entités

In [10]:
# Calculer le nombre de tokens pour chaque texte
train_df['token_count'] = train_df['text'].apply(lambda x: len(tokenizer(x)['input_ids']))

# Filtrer et afficher les textes avec plus de 512 tokens
long_texts = train_df[train_df['token_count'] > 512]
for index, row in long_texts.iterrows():
    print(f"Texte dépassant 512 tokens (index {index}): {row['text']}")

# Calculer le nombre et la proportion des textes dépassant 512 tokens
num_long_texts = len(long_texts)
total_texts = len(train_df)
proportion = num_long_texts / total_texts
# Calculer le nombre de tokens moyen par texte
average_token_count = train_df['token_count'].mean()

# Afficher les résultats
print(f"Nombre de textes dépassant 512 tokens : {num_long_texts}")
print(f"Proportion par rapport au DataFrame entier : {proportion:.2%}")
print(f"Nombre moyen de tokens par texte : {average_token_count:.2f}")


Nombre de textes dépassant 512 tokens : 0
Proportion par rapport au DataFrame entier : 0.00%
Nombre moyen de tokens par texte : 244.67


#### Check combinations

In [19]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)
from utils.ontology import *

In [20]:
train_df['relations'].loc[181]

[[0, 'STARTED_IN', 9],
 [7, 'IS_LOCATED_IN', 9],
 [5, 'IS_LOCATED_IN', 10],
 [1, 'GENDER_FEMALE', 1],
 [12, 'IS_LOCATED_IN', 10],
 [0, 'IS_LOCATED_IN', 10],
 [1, 'IS_LOCATED_IN', 10],
 [13, 'IS_LOCATED_IN', 9],
 [14, 'IS_LOCATED_IN', 10],
 [1, 'HAS_CATEGORY', 18],
 [5, 'IS_LOCATED_IN', 9],
 [9, 'IS_LOCATED_IN', 10],
 [14, 'IS_DEAD_ON', 15],
 [7, 'HAS_CATEGORY', 19],
 [12, 'IS_LOCATED_IN', 9],
 [0, 'IS_LOCATED_IN', 9],
 [7, 'GENDER_MALE', 7],
 [11, 'IS_LOCATED_IN', 10],
 [1, 'IS_LOCATED_IN', 9],
 [14, 'IS_LOCATED_IN', 9],
 [14, 'HAS_CATEGORY', 20],
 [0, 'STARTED_IN', 10],
 [0, 'START_DATE', 15],
 [7, 'IS_LOCATED_IN', 10],
 [0, 'END_DATE', 15],
 [14, 'GENDER_MALE', 14],
 [12, 'IS_OF_SIZE', 22],
 [13, 'IS_LOCATED_IN', 10],
 [1, 'IS_LOCATED_IN', 11],
 [13, 'IS_OF_SIZE', 21]]

In [26]:
possible_triples = set()

for i, row in train_df.iterrows():
    for relation in row['relations']:
        id_entity_subject, relation_type, id_entity_object = relation
        type_entity_subject = [entity['type'] for entity in row['entities'] if entity['id'] == id_entity_subject][0]
        type_entity_object = [entity['type'] for entity in row['entities'] if entity['id'] == id_entity_object][0]
        parent_entity_subject = TYPE_TO_PARENT.get(type_entity_subject)
        parent_entity_object = TYPE_TO_PARENT.get(type_entity_object)
        possible_triples.add((parent_entity_subject, relation_type, parent_entity_object))

In [30]:
for triple in possible_triples:
    if triple not in VALID_COMBINATIONS:
        print(triple)

('PERSON', 'HAS_CATEGORY', 'CATEGORY')
('PERSON', 'RESIDES_IN', 'PLACE')
('PLACE', 'HAS_CONTROL_OVER', 'MATERIEL')
('PERSON', 'GENDER_MALE', 'PERSON')
('ORG', 'IS_IN_CONTACT_WITH', 'ORG')
('ORG', 'OPERATES_IN', 'PLACE')
('PERSON', 'CREATED', 'MATERIEL')
('PLACE', 'HAS_LATITUDE', 'COORDINATE')
('EVENT', 'END_DATE', 'TIME')
('ORG', 'IS_OF_SIZE', 'QUANTITY')
('ORG', 'WAS_DISSOLVED_IN', 'TIME')
('EVENT', 'STARTED_IN', 'PLACE')
('ORG', 'IS_IN_CONTACT_WITH', 'PERSON')
('EVENT', 'IS_LOCATED_IN', 'PLACE')
('MATERIEL', 'WAS_CREATED_IN', 'TIME')
('ORG', 'HAS_CONTROL_OVER', 'PLACE')
('ORG', 'INITIATED', 'ORG')
('PLACE', 'IS_LOCATED_IN', 'PLACE')
('ORG', 'HAS_CONTROL_OVER', 'MATERIEL')
('EVENT', 'START_DATE', 'TIME')
('ORG', 'INITIATED', 'EVENT')
('PERSON', 'CREATED', 'ORG')
('PERSON', 'IS_IN_CONTACT_WITH', 'ORG')
('PERSON', 'IS_OF_NATIONALITY', 'NATONALITY')
('PERSON', 'HAS_CONTROL_OVER', 'PLACE')
('PERSON', 'HAS_CONTROL_OVER', 'MATERIEL')
('PERSON', 'IS_IN_CONTACT_WITH', 'PERSON')
('ORG', 'IS_OF

In [8]:
max_nb_entities_in_example = 0
for i, row in train_df.iterrows():
    nb_entities = 0
    for entity in row['entities']:
        nb_entities += len(entity['mentions'])
    if nb_entities > max_nb_entities_in_example:
        max_nb_entities_in_example = nb_entities
print(max_nb_entities_in_example)

0
33
39
50
68


In [6]:
train_df.loc[181]['entities']

[{'id': 0,
  'mentions': [{'value': 'accident', 'start': 70, 'end': 78},
   {'value': 'accident de circulation', 'start': 100, 'end': 123},
   {'value': 'accident', 'start': 275, 'end': 283}],
  'type': 'ACCIDENT'},
 {'id': 1,
  'mentions': [{'value': 'Anam Destresse', 'start': 0, 'end': 14},
   {'value': 'Anam Destresse', 'start': 431, 'end': 445}],
  'type': 'CIVILIAN'},
 {'id': 2,
  'mentions': [{'value': 'moto', 'start': 148, 'end': 152},
   {'value': 'moto', 'start': 550, 'end': 554}],
  'type': 'MATERIEL'},
 {'id': 3,
  'mentions': [{'value': 'bus', 'start': 159, 'end': 162},
   {'value': 'bus', 'start': 208, 'end': 211},
   {'value': 'bus', 'start': 589, 'end': 592}],
  'type': 'MATERIEL'},
 {'id': 4,
  'mentions': [{'value': 'panneaux de signalisation',
    'start': 289,
    'end': 314}],
  'type': 'MATERIEL'},
 {'id': 5,
  'mentions': [{'value': 'blessés', 'start': 470, 'end': 477}],
  'type': 'GROUP_OF_INDIVIDUALS'},
 {'id': 6,
  'mentions': [{'value': 'hélicoptère', 'start':